<a href="https://colab.research.google.com/github/HawkDukez/W4995DL-Final/blob/main/FID_IS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "drive/MyDrive/DL_final/save_500.zip" -d  "drive/MyDrive/DL_final/"

Archive:  drive/MyDrive/DL_final/save_500.zip
  inflating: drive/MyDrive/DL_final/save_real500.npy  
  inflating: drive/MyDrive/DL_final/save_fake500.npy  


In [3]:
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import cifar10
from skimage.transform import resize
from numpy import asarray
import numpy
import torch.nn.functional as F
import torch
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		new_image = resize(image, new_shape, 0)
		images_list.append(new_image)
	return asarray(images_list)
 

def calculate_inception_score(images, n_split=10, eps=1E-16):
	model = InceptionV3()
	processed = images.astype('float32')


	processed = scale_images(processed,(299,299,3))

	processed = preprocess_input(processed)
	yhat = model.predict(processed)
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		ix_start, ix_end = i * n_part, i * n_part + n_part
		p_yx = yhat[ix_start:ix_end]

		p_y = expand_dims(p_yx.mean(axis=0), 0)
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
		sum_kl_d = kl_d.sum(axis=1)
		avg_kl_d = mean(sum_kl_d)
		is_score = exp(avg_kl_d)
		scores.append(is_score)
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std
#def preprocess1(img):
#    return F.interpolate(img, size=(299,299), mode='bilinear', align_corners=False)
images=numpy.load('drive/MyDrive/DL_final/save_fake500.npy')

shuffle(images)
print(type(images))
print('loaded', images.shape)
is_avg, is_std = calculate_inception_score(images)
print('score', is_avg, is_std)

<class 'numpy.ndarray'>
loaded (500, 3, 256, 256)
96124928/96112376 [==============================] - 0s 0us/step
score 1.0038469 0.00215613


In [4]:
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

from skimage.transform import resize

 
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		new_image = resize(image, new_shape, 0)
		images_list.append(new_image)
	return asarray(images_list)
 
def calculate_fid(model, images1, images2):
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	covmean = sqrtm(sigma1.dot(sigma2))
	if iscomplexobj(covmean):
		covmean = covmean.real
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid
 
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

images1 = numpy.load('drive/MyDrive/DL_final/save_real500.npy')
shuffle(images1)

images2 = numpy.load('drive/MyDrive/DL_final/save_fake500.npy')
print(type(images1))
print('Loaded', images1.shape, images2.shape)

images1 = images1.astype('float32')
images2 = images2.astype('float32')
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)

images1 = preprocess_input(images1)
images2 = preprocess_input(images2)

fid = calculate_fid(model, images1, images2)
print('FID: %.3f' % fid)

87924736/87910968 [==============================] - 0s 0us/step
<class 'numpy.ndarray'>
Loaded (500, 3, 256, 256) (500, 3, 256, 256)
Scaled (500, 299, 299, 3) (500, 299, 299, 3)
FID: 0.029
